<a href="https://colab.research.google.com/github/bartaLevente/nlp/blob/main/Finetune_phi15_codealpaca_Barta_Levente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "microsoft/phi-1.5"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
prompt = (
    "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n"
    "### Instruction:\n"
    "Create a Python list comprehension to get the squared values of a list [1, 2, 3, 5, 8, 13].\n"
    "### Input:\n"
    "\n\n"
    "### Output:\n"
)

input_ids = tokenizer.encode(prompt, return_tensors="pt", max_length=512, truncation=True)

# Generate code
output_ids = model.generate(
    input_ids,
    max_length=200,
    num_return_sequences=1,
    num_beams=4,
    early_stopping=True
)

# Decode the generated code
generated_code = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(generated_code)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Create a Python list comprehension to get the squared values of a list [1, 2, 3, 5, 8, 13].
### Input:


### Output:

```python
[1, 4, 9, 25, 64, 169]
```

#### Solution:

```python
numbers = [1, 2, 3, 5, 8, 13]
squared_numbers = [number ** 2 for number in numbers]
print(squared_numbers)
```

### Exercise 2:

Create a Python list comprehension to get the even numbers from a list [1, 2, 3, 4, 5, 6, 7, 8, 9, 10].
### Input:


### Output:

```python
[2, 4, 6,


Az alapmodell 3 perces futási idővel ezt a választ produkálta. A generált kód része működik, de utána a modell hallucinálni kezd.


In [ ]:
!pip install bitsandbytes peft accelerate transformers wandb datasets

In [ ]:
from datasets import load_dataset
import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

ds = load_dataset("iamtarun/python_code_instructions_18k_alpaca")
tran_dataset = ds['train'].select(range(6000))
print(tran_dataset[0])
print(tran_dataset)

README.md:   0%|          | 0.00/905 [00:00<?, ?B/s]

(…)-00000-of-00001-8b6e212f3e1ece96.parquet:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18612 [00:00<?, ? examples/s]

{'instruction': 'Create a function to calculate the sum of a sequence of integers.', 'input': '[1, 2, 3, 4, 5]', 'output': '# Python code\ndef sum_sequence(sequence):\n  sum = 0\n  for num in sequence:\n    sum += num\n  return sum', 'prompt': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCreate a function to calculate the sum of a sequence of integers.\n\n### Input:\n[1, 2, 3, 4, 5]\n\n### Output:\n# Python code\ndef sum_sequence(sequence):\n  sum = 0\n  for num in sequence:\n    sum += num\n  return sum'}
Dataset({
    features: ['instruction', 'input', 'output', 'prompt'],
    num_rows: 6000
})


Egy instruction dataset első 6000 sorával finomhangoltam az eredeti modellt


In [ ]:
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    formatted_examples = []

    for instruction, input_text, output in zip(examples["instruction"], examples["input"], examples["output"]):
        # Format as a single string with clear markers
        formatted_text = (
            f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n"
            f"### Instruction:\n{instruction}\n\n"
            f"### Input:\n{input_text}\n\n"
            f"### Output:\n{output}"
        )
        formatted_examples.append(formatted_text)

    # Tokenize with padding and truncation
    tokenized_inputs = tokenizer(
        formatted_examples,
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

    # Process for causal language modeling
    input_ids = tokenized_inputs["input_ids"].clone()
    labels = input_ids.clone()

    # For each example, find the position where "### Output:" starts
    for idx, example in enumerate(formatted_examples):
        # Find the position of "### Output:" in tokens
        output_text = "### Output:"
        output_tokens = tokenizer.encode(output_text, add_special_tokens=False)

        # Search for output_tokens in input_ids
        input_ids_list = input_ids[idx].tolist()

        # Find the start position of the output section
        start_pos = -1
        for i in range(len(input_ids_list) - len(output_tokens) + 1):
            if input_ids_list[i:i+len(output_tokens)] == output_tokens:
                # Add len(output_tokens) to point to the actual output content
                start_pos = i + len(output_tokens)
                break

        # If found, mask the instruction and input part with -100 for the labels
        if start_pos > 0:
            labels[idx, :start_pos] = -100

    # Return the tokenized dataset with prepared labels
    return {
        "input_ids": input_ids,
        "attention_mask": tokenized_inputs["attention_mask"],
        "labels": labels
    }

# Process the subset
print("Tokenizing dataset...")
tokenized_dataset = tran_dataset.map(
    tokenize_function,
    batched=True,
    batch_size=8,
    remove_columns=tran_dataset.column_names,
    desc="Tokenizing"
)

Tokenizing dataset...


Tokenizing:   0%|          | 0/6000 [00:00<?, ? examples/s]

A dataset tokenizálása


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
from peft import LoraConfig, prepare_model_for_kbit_training
import torch

model_name = "microsoft/phi-1.5"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,)

model_4bit = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    use_cache = False,
    device_map={'':0}
)

In [ ]:
import bitsandbytes as bnb
target = bnb.nn.Linear4bit
cls = bnb.nn.Linear4bit
modules = {
        name.split('.')[-1]
        for name, module in model_4bit.named_modules()
        if isinstance(module, cls)
    }
print(modules)

{'fc2', 'dense', 'fc1', 'k_proj', 'q_proj', 'v_proj'}


In [ ]:
model_4bit

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2048)
    (layers): ModuleList(
      (0-23): 24 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (dense): Linear4bit(in_features=2048, out_features=2048, bias=True)
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear4bit(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear4bit(in_features=8192, out_features=2048, bias=True)
        )
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (rotary_emb): PhiRotaryEmbedding()
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (final_layernorm): La

In [ ]:
import torch, os
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer, Trainer
from peft import LoraConfig
from transformers import TrainingArguments
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training
)


model_4bit = prepare_model_for_kbit_training(model_4bit)


lora_alpha = 16
lora_dropout = 0.1
lora_r = 32
lora_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['v_proj', 'k_proj', 'q_proj']
)

peft_model = get_peft_model(model_4bit, lora_config)



In [ ]:


output_dir = "./phi_lora_temp"
per_device_train_batch_size = 8
gradient_accumulation_steps = 4
optim = 'paged_adamw_8bit'
save_steps = 1000
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
warmup_ratio = 0.1
lr_scheduler_type = "cosine"
training_arguments = TrainingArguments(
    output_dir=output_dir,
    push_to_hub=True,
    hub_model_id="bartalevente12/phi-lora-finetuned",
    run_name='unique_run_name',
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    group_by_length=False,
    lr_scheduler_type=lr_scheduler_type,
    label_names=['labels']
)

import wandb
wandb.init(mode='disabled')


trainer = Trainer(
    model=peft_model,
    args=training_arguments,
    train_dataset=tokenized_dataset,
)

In [ ]:
# Not sure if needed but noticed this in https://colab.research.google.com/drive/1t3exfAVLQo4oKIopQT1SKxK4UcYg7rC1#scrollTo=7OyIvEx7b1GT
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

torch.cuda.empty_cache()
# Train :)
trainer.train() # resume_from_checkpoint = True

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,6.004600
2,5.914200
3,5.757800
4,5.428200
5,5.288000
6,5.590300
7,5.457100
8,5.909100
9,5.249700
10,6.000100


TrainOutput(global_step=561, training_loss=0.4678832860978954, metrics={'train_runtime': 7684.502, 'train_samples_per_second': 2.342, 'train_steps_per_second': 0.073, 'total_flos': 7.282322671927296e+16, 'train_loss': 0.4678832860978954, 'epoch': 2.986666666666667})

In [ ]:
trainer.save_model("ft-phi-codepaca")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
import torch

base_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-1.5",
    trust_remote_code=True,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1.5")
tokenizer.pad_token = tokenizer.eos_token

# Load the PEFT configuration and apply the LoRA weights to the base model
peft_model_path = "bartalevente12/phi-lora-finetuned"
config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)  # Apply LoRA adapter to base model

# Format prompt exactly like during training
prompt = (
    "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n"
    "### Instruction:\n"
    "Create a Python list comprehension to get the squared values of a list [1, 2, 3, 5, 8, 13].\n"
    "### Input:\n"
    "\n\n"
    "### Output:\n"
)

#Tokenize the prompt
input_ids = tokenizer.encode(prompt, return_tensors="pt", max_length=512, truncation=True)
input_ids = input_ids.to(model.device)

#Generate the response
output_ids = model.generate(
    input_ids,
    max_length=300,
    num_return_sequences=1,
    temperature=0.3,
    top_p=0.9,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(generated_text)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/736 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/37.8M [00:00<?, ?B/s]

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Create a Python list comprehension to get the squared values of a list [1, 2, 3, 5, 8, 13].
### Input:


### Output:
squared_values = [x**2 for x in [1, 2, 3, 5, 8, 13]]


A finomhangolt verzió 10 másodpercen belül hallucináció néküli, helyes választ ad :)